# Finetune Pix2Struct model on Pix2Code dataset

## Setup Envirnoment

In [ ]:
!pip install --upgrade git+https://github.com/huggingface/transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-g6ohb2pm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-g6ohb2pm
  Resolved https://github.com/huggingface/transformers to commit e26c6f03be35a621d26d79ae59e21ceac3ffa73e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.8 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7170473 sha256=632113f666ee1ea335c9c8923693ed530eb78473ba4e07cf488ab39e22f4b112
  Stored in directory: /tmp/pip-ephem-w

In [ ]:
!pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00


## Import necessary libraries

In [ ]:
from google.colab import drive
import os
from pathlib import Path
import zipfile
import re
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.utils.data import random_split
from torch.nn import functional as F
from transformers import Pix2StructForConditionalGeneration, AutoProcessor
from transformers.optimization import Adafactor, get_cosine_schedule_with_warmup
from nltk import edit_distance
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
import wandb
from tqdm import tqdm

## Define variables and parameters

In [ ]:
G_DRIVE_FOLDER_DATASET = '/content/drive/MyDrive/Datasets/'
G_DRIVE_FOLDER_CHECKPOINTS = '/content/drive/MyDrive/Checkpoints/'
DATASET_NAME = 'pix2code_web'
ZIP_NAME = DATASET_NAME + '.zip'
DESTINATION_FOLDER= '/content/data/'
DATASET_FOLDER = DESTINATION_FOLDER + DATASET_NAME

MAX_SENTENCE_LEN = 250
MAX_NEW_TOKENS = MAX_SENTENCE_LEN
MAX_PATCHES = 1024

DEBUG = False
VERBOSE = True

BATCH_SIZE = 1
NUM_WARMUP_STEPS = 5000
MAX_EPOCHS = 20
LR = 1e-4
CHECK_VAL_EVERY_N_EPOCH = 5
GRADIENT_CLIP_VAL = 1.0
ACCUMULATE_GRAD_BATCHES = 8 / BATCH_SIZE


## Load Pix2Code Dataset

### Mount Google Drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


### Import zip file from Google Drive

In [ ]:
os.makedirs(DESTINATION_FOLDER, exist_ok=True)

with zipfile.ZipFile(G_DRIVE_FOLDER_DATASET + ZIP_NAME, "r") as zf:
  zf.extractall(DESTINATION_FOLDER)

## Load Model and Processor

In [ ]:
repo_id = "google/pix2struct-base"

processor = AutoProcessor.from_pretrained(repo_id)
model = Pix2StructForConditionalGeneration.from_pretrained(repo_id, is_encoder_decoder=True)

## Create Dataset class

In [ ]:
def preprocess_gui(gui_text):
    text_cleaned = gui_text.replace('\n', ' ')
    text_cleaned_without_multiple_spaces = re.sub(r'\s+', ' ', text_cleaned)
    return text_cleaned_without_multiple_spaces

### Find max sentence length in the whole database

In [ ]:
# Find max length
max_length = 0

# Get a list of all files in root_dir
files = os.listdir(DATASET_FOLDER)

# Find only gui files
gui_files = [file for file in files if file.endswith('.gui')]

for gui_file_path in gui_files:
    with open(DATASET_FOLDER + "/" + gui_file_path, "r") as reader:
        splitted_text = processor.tokenizer(preprocess_gui(reader.read())).tokens()
        if len(splitted_text) > max_length:
            max_length = len(splitted_text)
print("max_length = ", max_length)

max_length =  200


In [ ]:
class Pix2CodeDataset(Dataset):
    def __init__(self, root_dir: str, transform):

        self.root_dir = root_dir
        self.transform = transform

        self.max_patches = MAX_PATCHES
        self.max_length = MAX_SENTENCE_LEN
        self.ignore_id = -100
        self.filenames = []

        # Get a list of all files in root_dir
        files = os.listdir(root_dir)

        # Filter out files that don't end with .png
        png_files = [file for file in files if file.endswith('.png')]

        # Build list of tuples with png and gui paths
        for png_file in png_files:
            gui_file = png_file.replace('.png', '.gui')
            if gui_file in files:
                self.filenames.append((os.path.join(root_dir, png_file), os.path.join(root_dir, gui_file)))

        # Read gui files and add new tokens to dictionary
        tokens_to_add = set()

        for _, gui_file_path in self.filenames:
          with open(gui_file_path, "r") as reader:
              splitted_text = processor.tokenizer(preprocess_gui(reader.read())).tokens()
              tokens_to_add = tokens_to_add.union(set(splitted_text))

        newly_added_num = processor.tokenizer.add_tokens(list(tokens_to_add))
        print("newly_added_num: ", newly_added_num)
        if newly_added_num > 0:
            model.decoder.resize_token_embeddings(len(processor.tokenizer))

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        image_path, text_path = self.filenames[idx]

        # Load image
        image = Image.open(image_path).convert('RGB')

        if DEBUG:
          image.show()

        if self.transform:
            image = self.transform(image)

        # Load text
        with open(text_path, 'r') as f:
            text = f.read()
            text_cleaned = preprocess_gui(text)

        if DEBUG:
          print("text:")
          print(text)
          print("text_cleaned:")
          print(text_cleaned)

        # prepare inputs
        encoding = processor(images=image, max_patches=self.max_patches, return_tensors="pt")
        encoding = {k:v.squeeze() for k,v in encoding.items()}


        # prepare targets
        input_ids = processor.tokenizer(
            text_cleaned,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        ).input_ids

        labels = input_ids.squeeze().clone()
        labels[labels == processor.tokenizer.pad_token_id] = self.ignore_id  # model doesn't need to predict pad token
        encoding["labels"] = labels
        return encoding, text_cleaned

In [ ]:
# Transformations for the image
transform = transforms.Compose([
    transforms.ToTensor(),  # convert PIL Image to PyTorch Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # normalize for pretrained models
])

# Instantiate the CustomDataset
dataset = Pix2CodeDataset(root_dir=DATASET_FOLDER, transform=transform)

# Determine lengths of splits
train_len = int(0.80 * len(dataset))
valid_len = int(0.10 * len(dataset))
test_len = len(dataset) - train_len - valid_len

# Perform the split
train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_len, valid_len, test_len])

# Use DataLoader for batching and shuffling
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

newly_added_num:  0


In [ ]:
batch = next(iter(train_dataloader))
encoding, text = batch

In [ ]:
encoding

{'flattened_patches': tensor([[[ 1.0000,  1.0000,  0.1288,  ...,  0.1288,  0.3797,  0.6750],
          [ 1.0000,  2.0000,  0.1288,  ..., -4.7502, -4.6082, -4.2908],
          [ 1.0000,  3.0000,  0.1288,  ..., -4.7502, -4.6082, -4.2908],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]]),
 'attention_mask': tensor([[1., 1., 1.,  ..., 0., 0., 0.]]),
 'labels': tensor([[ 9142,   435, 17103,   288,   549,  8225,   275, 17103,   288,   549,
           8225,   275, 17103,   288,  8225,   275, 17103,   288,   549,  8225,
            512,  3459,   435, 20391, 45568,  1930,   435,   664,   288,  2681,
            275,  1280,   275, 17103,   288,  9549,   512, 20391, 45568,  1930,
            435,   664,   288,  2681,   275,  1280,   275, 17103,   288,  5596,
            512, 20391, 45568,  1930,   435,   664,   288,

In [ ]:
encoding["flattened_patches"][0]

tensor([[ 1.0000,  1.0000,  0.1288,  ...,  0.1288,  0.3797,  0.6750],
        [ 1.0000,  2.0000,  0.1288,  ..., -4.7502, -4.6082, -4.2908],
        [ 1.0000,  3.0000,  0.1288,  ..., -4.7502, -4.6082, -4.2908],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [ ]:
text

('header { btn-inactive, btn-inactive, btn-active, btn-inactive } row { quadruple { small-title, text, btn-green } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-green } } row { single { small-title, text, btn-green } } row { double { small-title, text, btn-red } double { small-title, text, btn-green } } ',)

In [ ]:
for k,v in encoding.items():
    print(k,v.shape)

flattened_patches torch.Size([1, 1024, 770])
attention_mask torch.Size([1, 1024])
labels torch.Size([1, 250])


## Training

In [ ]:
def move_to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [move_to_device(x, device) for x in data]
    elif isinstance(data, dict):
        return {k: move_to_device(v, device) for k, v in data.items()}
    elif isinstance(data, torch.Tensor):
        return data.to(device)
    else:
        return data

In [ ]:
def train_model(config, processor, model):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    optimizer = Adafactor(model.parameters(), scale_parameter=False, relative_step=False, lr=config.get("lr"), weight_decay=1e-05)

    # Use total steps (i.e., max_epochs * length_of_train_data)
    total_steps = config.get("max_epochs") * len(train_dataloader)
    scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                num_warmup_steps=config.get("num_warmup_steps"),
                                                num_training_steps=total_steps)


    wandb.init(project="Pix2Struct", name="run-pix2struct-pix2code", config=config)

    global_step = 0  # to keep track of total steps

    num_epochs = config.get("max_epochs")
    for epoch in range(num_epochs):
        model.train()

        train_loop = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch}/{num_epochs - 1} - train loop")

        for step, batch in train_loop:
            encoding, _ = batch
            encoding = move_to_device(encoding, device)
            outputs = model(**encoding)
            loss = outputs.loss
            loss.backward()

            if global_step % config.get('accumulate_grad_batches', 1) == 0:
                optimizer.step()
                optimizer.zero_grad()

            if config.get("gradient_clip_val"):
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.get("gradient_clip_val"))

            # Update the progress bar
            train_loop.set_postfix(loss=loss.item())

            scheduler.step()
            global_step += 1

        if epoch  == 0 or epoch == num_epochs - 1 or (epoch + 1) % config.get("check_val_every_n_epoch") == 0:
            model.eval()
            ed_scores = []
            bleu_scores = []
            with torch.no_grad():

                val_loop = tqdm(enumerate(val_dataloader), total=len(val_dataloader), desc=f"Epoch {epoch}/{num_epochs - 1} - valid loop")
                for i, batch in val_loop:
                    encoding, answers = batch
                    encoding = move_to_device(encoding, device)
                    answers = move_to_device(answers, device)
                    flattened_patches, attention_mask = encoding["flattened_patches"], encoding["attention_mask"]

                    outputs = model.generate(flattened_patches=flattened_patches,
                                            attention_mask=attention_mask,
                                            max_new_tokens=MAX_NEW_TOKENS,
                                            return_dict_in_generate=True,)

                    predictions = []
                    for seq in processor.tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True):
                        predictions.append(seq)

                    for pred, answer in zip(predictions, answers):
                        ed_score = edit_distance(pred, answer) / max(len(pred), len(answer))
                        ed_scores.append(ed_score)

                        bleu_score = corpus_bleu([[answer]], [pred], smoothing_function=SmoothingFunction().method4)
                        bleu_scores.append(bleu_score)

                        # Update the progress bar
                        val_loop.set_postfix(val_bleu_score=np.mean(bleu_scores))

                        if config.get("verbose", False):
                            print(f"\nPrediction: {pred}")
                            print(f"    Answer: {answer}")
                            print(f" Normed ED: {ed_score:.2f}")
                            print(f"      Bleu: {bleu_score:.2f}\n")

                avg_ed_score = np.mean(ed_scores)
                avg_bleu_score = np.mean(bleu_scores)

                # Save the model after each validation step
                model_name = f"model_epoch[{epoch}]_bleu[{avg_bleu_score:.2f}].pth"
                torch.save(model.state_dict(), G_DRIVE_FOLDER_CHECKPOINTS + model_name)

        if config.get("verbose", False):
            print(f"\n        Train Loss: {loss.item():.4f}")
            print(f"Val Avg Bleu Score: {avg_bleu_score:.2f}")
            print(f" Val Avg Normed ED: {avg_ed_score:.2f}\n")
        wandb.log({"train_loss": loss.item(), "val_bleu": avg_bleu_score, "val_ed": avg_ed_score, **{f'lr_{i}': param_group['lr'] for i, param_group in enumerate(optimizer.param_groups)}})

    wandb.finish()

In [ ]:
config = {
          "batch_size": BATCH_SIZE,
          "num_warmup_steps": NUM_WARMUP_STEPS,
          "max_epochs": MAX_EPOCHS,
          "lr": LR,
          "check_val_every_n_epoch": CHECK_VAL_EVERY_N_EPOCH,
          "gradient_clip_val": GRADIENT_CLIP_VAL,
          "accumulate_grad_batches": ACCUMULATE_GRAD_BATCHES,
          "verbose": VERBOSE,
}

In [ ]:
train_model(config, processor, model)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch 0/19 - valid loop:   1%|          | 1/174 [00:09<26:37,  9.23s/it, val_bleu_score=0.00929]


Prediction: <<img_src=1386886288_1_thumb img_alt=Qfcxo> <img_src=1386886288_2_thumb img_alt=Qfcxo> <img_src=1386886288_3_thumb img_alt=Qfcxo> <img_src=1386886288_4_thumb img_alt=Qfcxo> <img_src=1386886288_5_thumb img_alt=Qfcxo> <img_src=1386886288_6_thumb img_alt=Qfcxo> <img_src=1386886288_7_thumb img_alt=Qfcxo> <img_src=1386886288_8_thumb img_alt=Qfcxo> <img_src=1386886288_1_thumb img_alt=Qfc
    Answer: header { btn-active, btn-inactive, btn-inactive, btn-inactive, btn-inactive } row { single { small-title, text, btn-orange } } row { double { small-title, text, btn-green } double { small-title, text, btn-orange } } row { quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-green } } 
 Normed ED: 0.93
      Bleu: 0.01



Epoch 0/19 - valid loop:   1%|          | 2/174 [00:13<18:16,  6.38s/it, val_bleu_score=0.0082]

...

Epoch 19/19 - valid loop:  99%|█████████▉| 173/174 [08:43<00:02,  2.47s/it, val_bleu_score=0.984]


Prediction: header { btn-inactive, btn-inactive, btn-active } row { quadruple { small-title, text, btn-green } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-orange } quadruple { small-title, text, btn-red } } row { single { small-title, text, btn-green } } row { double { small-title, text, btn-green } double { small-title, text, btn-red } } 
    Answer: header { btn-inactive, btn-inactive, btn-active } row { quadruple { small-title, text, btn-green } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-orange } quadruple { small-title, text, btn-red } } row { single { small-title, text, btn-green } } row { double { small-title, text, btn-green } double { small-title, text, btn-red } } 
 Normed ED: 0.00
      Bleu: 1.00



Epoch 19/19 - valid loop: 100%|██████████| 174/174 [08:46<00:00,  3.03s/it, val_bleu_score=0.983]



Prediction: header { btn-active, btn-inactive, btn-inactive } row { single { small-title, text, btn-red } } row { double { small-title, text, btn-green } double { small-title, text, btn-orange } } row { quadruple { small-title, text, btn-orange } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-orange } quadruple { small-title, text, btn-orange } } 
    Answer: header { btn-active, btn-inactive } row { single { small-title, text, btn-red } } row { double { small-title, text, btn-green } double { small-title, text, btn-orange } } row { quadruple { small-title, text, btn-orange } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-orange } quadruple { small-title, text, btn-orange } } 
 Normed ED: 0.04
      Bleu: 0.96


        Train Loss: 0.0145
Val Avg Bleu Score: 0.98
 Val Avg Normed ED: 0.02



lr_0,▃▅▇███▇▇▆▆▅▄▄▃▂▂▂▁▁▁
train_loss,█▃▃▃▂▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁
val_bleu,▁▁▁▁▆▆▆▆▆███████████
val_ed,████▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁
lr_0,0.0
train_loss,0.01446
val_bleu,0.98345
val_ed,0.01642


## Test the model

### Removing all spaces from output sentences

In [ ]:
def test_model(config, processor, model):
    # Testing on test set
    model.eval()

    ed_scores = []
    bleu_scores = []
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    with torch.no_grad():

        test_loop = tqdm(enumerate(test_dataloader), total=len(test_dataloader), desc=f"Test loop")
        for i, batch in test_loop:
            encoding, answers = batch
            encoding = move_to_device(encoding, device)
            answers = move_to_device(answers, device)
            flattened_patches, attention_mask = encoding["flattened_patches"], encoding["attention_mask"]

            outputs = model.generate(flattened_patches=flattened_patches, attention_mask=attention_mask, max_new_tokens=MAX_NEW_TOKENS, return_dict_in_generate=True)

            predictions = []

            for seq in processor.tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True):
                predictions.append(seq)

            for pred, answer in zip(predictions, answers):
                ed_score = edit_distance(pred, answer) / max(len(pred), len(answer))
                ed_scores.append(ed_score)

                bleu_score = corpus_bleu([[answer]], [pred], smoothing_function=SmoothingFunction().method4)
                bleu_scores.append(bleu_score)


                # Update the progress bar
                test_loop.set_postfix(test_bleu_score=np.mean(bleu_scores))

                if config.get("verbose", False):
                    print(f"\nPrediction: {pred}")
                    print(f"    Answer: {answer}")
                    print(f" Normed ED: {ed_score:.2f}")
                    print(f"      Bleu: {bleu_score:.2f}")

                    avg_ed_score = np.mean(ed_scores)
                    avg_bleu_score = np.mean(bleu_scores)

        print("\n\nRESULTS:")
        print(f"Test Avg Bleu Score : {avg_bleu_score:.2f}")
        print(f"Test Avg Normed ED  : {avg_ed_score:.2f}")

In [ ]:
test_model(config, processor, model)

Test loop:   1%|          | 1/175 [00:03<09:29,  3.27s/it, test_bleu_score=1]


Prediction: header { btn-inactive, btn-inactive, btn-active } row { double { small-title, text, btn-red } double { small-title, text, btn-orange } } row { single { small-title, text, btn-orange } } row { quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-green } quadruple { small-title, text, btn-orange } } 
    Answer: header { btn-inactive, btn-inactive, btn-active } row { double { small-title, text, btn-red } double { small-title, text, btn-orange } } row { single { small-title, text, btn-orange } } row { quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-green } quadruple { small-title, text, btn-orange } } 
 Normed ED: 0.00
      Bleu: 1.00


Test loop:   1%|          | 2/175 [00:06<09:42,  3.37s/it, test_bleu_score=1]


Prediction: header { btn-inactive, btn-active, btn-inactive } row { double { small-title, text, btn-orange } double { small-title, text, btn-orange } } row { single { small-title, text, btn-orange } } row { quadruple { small-title, text, btn-green } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-orange } quadruple { small-title, text, btn-green } } 
    Answer: header { btn-inactive, btn-active, btn-inactive } row { double { small-title, text, btn-orange } double { small-title, text, btn-orange } } row { single { small-title, text, btn-orange } } row { quadruple { small-title, text, btn-green } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-orange } quadruple { small-title, text, btn-green } } 
 Normed ED: 0.00
      Bleu: 1.00


Test loop:   2%|▏         | 3/175 [00:08<07:25,  2.59s/it, test_bleu_score=1]

...


Prediction: header { btn-inactive, btn-inactive, btn-active } row { quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-green } quadruple { small-title, text, btn-red } } row { single { small-title, text, btn-red } } row { double { small-title, text, btn-green } double { small-title, text, btn-green } } 
    Answer: header { btn-inactive, btn-inactive, btn-active } row { quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-green } quadruple { small-title, text, btn-red } } row { single { small-title, text, btn-red } } row { double { small-title, text, btn-green } double { small-title, text, btn-green } } 
 Normed ED: 0.00
      Bleu: 1.00


Test loop:  99%|█████████▉| 174/175 [08:31<00:03,  3.14s/it, test_bleu_score=0.984]


Prediction: header { btn-active, btn-inactive } row { single { small-title, text, btn-orange } } row { quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-orange } quadruple { small-title, text, btn-red } } row { double { small-title, text, btn-orange } double { small-title, text, btn-orange } } 
    Answer: header { btn-active, btn-inactive } row { single { small-title, text, btn-orange } } row { quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-orange } quadruple { small-title, text, btn-red } } row { double { small-title, text, btn-orange } double { small-title, text, btn-orange } } 
 Normed ED: 0.00
      Bleu: 1.00


Test loop: 100%|██████████| 175/175 [08:34<00:00,  2.94s/it, test_bleu_score=0.984]


Prediction: header { btn-inactive, btn-inactive, btn-inactive, btn-inactive, btn-active } row { double { small-title, text, btn-orange } double { small-title, text, btn-green } } row { quadruple { small-title, text, btn-green } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-orange } } row { single { small-title, text, btn-green } } 
    Answer: header { btn-inactive, btn-inactive, btn-inactive, btn-active } row { double { small-title, text, btn-orange } double { small-title, text, btn-green } } row { quadruple { small-title, text, btn-green } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-red } quadruple { small-title, text, btn-orange } } row { single { small-title, text, btn-green } } 
 Normed ED: 0.04
      Bleu: 0.96


RESULTS:
Test Avg Bleu Score : 0.98
Test Avg Normed ED  : 0.02
